.. meta::
    :description:  An implementation of a progressivly interactive NSGA-II algorithm that uses user preference to guide the solving of multi=objective optimization problems. Value function optimization is used to take a user generated ranking of key points to segment the search space.

.. meta::
    :keywords: PINSGA2, PINSGA-II, Progressivly Interactive, Multi-objective Optimization, Python

# PI-NSGA-II: Progressivly Interactive Optimization Using NSGA-II

PI-NSGA-II (PINSGA2) is an extension of the NSGA-II algorithm that introduces a custom domination operator that uses user preference to guide the optimization process in multi-objective problems. 

At regular intervals (tau generations), the algorithm pauses to select eta sparse points from the current Pareto front. These points are given to the decision-maker, who ranks them based on personal preference. The ranking can be either absolute, where solutions are ranked from highest to lowest preference, or pairwise, where solutions are compared in pairs to establish a relative preference order.

The user-provided rankings are then used to fit a value function that splits the search space of the problem. These regions modify the domination of points for the next tau generations, and then the process is repeated.

This algorithm can be customized to fit your needs in the following ways:

- Number of gens between each fit (tau)
- Number of pts to rank (eta)

- Optimization Method (opt_method):
  - 'trust-constr'
  - 'SLSQP'
  - 'ES'
  - 'GA'

- Value Function Type (vf_type):
  - 'linear' - will only segment the search space with parallel lines
  - 'poly' - will segment the search space with iso-curves

- Ranking Method (ranking_method):
  - 'absolute' - User must provide a complete ranking
  - 'pairwise' - User only compares two solutions at time

- Automatic Ranking (automated_dm):
  - an implementation of the automatedDM class cutomized to your need, if used ranking_method is ignored

#### Example

this is a simple example with a custom automatedDM. This is using problem ztd3, for it recognizable steps in the pareto-front. The decision maker is set to rank four points every 10 generations based on their distance from the center of the second region of the solution. In this case imagine you only want solution in this area of the search space, other solutions are undisireable or unfeasable, it is better to explore what you know you want to find more options.

In [3]:
from pymoo.algorithms.moo.pinsga2 import PINSGA2, AutomatedDM
from pymoo.problems import get_problem
from pymoo.optimize import minimize
import numpy as np
from pymoo.visualization.scatter import Scatter


class simpleDM( AutomatedDM ):
    
    def makeDecision(F): 
        # Euclidean distance from center of region # 2
        distances = np.sqrt( ( F[:, 0] - 0.21999557 )**2 + ( 0.40167775 - F[:, 1] )**2 )  

        ranks = distances.argsort().argsort() + 1
        return ranks

problem = get_problem("zdt3")

algorithm = PINSGA2(pop_size=30, 
                    tau=10,
                    eta=4,
                    opt_method="trust-constr", 
                    vf_type="poly",
                    automated_dm=simpleDM)

res = minimize(problem,
               algorithm,
               ('n_gen', 100),
               seed=1,
               verbose=False)

plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="red")
plot.show()

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3 and the array at index 1 has size 4

### API